In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# === VERİ YÜKLE ===
df = pd.read_excel("EMT_new_data.xlsx")
df['date'] = pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month
df['weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)
df['month_start'] = df['date'].dt.is_month_start.astype(int)

df = pd.get_dummies(df, drop_first=True)
df = df.drop(columns=["date"], errors="ignore")
df = df.astype(float)

# === Hedef ve Girdi ===
y = df["orders"]
X = df.drop(columns=["orders"])

# === Train-Test Böl
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Model Eğit
order_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
order_model.fit(X_train, y_train)

# === Kaydet: Model
with open("order_model.pkl", "wb") as f:
    pickle.dump(order_model, f)

# === Kaydet: Kullanılan Feature'lar
with open("order_model_features.pkl", "wb") as f:
    pickle.dump(list(X_train.columns.tolist()), f)


print("✅ order_model ve order_model_features başarıyla kaydedildi.")


✅ order_model ve order_model_features başarıyla kaydedildi.


In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# ---------------------------
# 1. Özellik mühendisliği fonksiyonu
# ---------------------------
def feature_engineering(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['weekday'] = df['date'].dt.day_name()
    df['is_weekday'] = df['weekday'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']).astype(int)
    df['weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)
    df['month_start'] = (df['date'].dt.day <= 3).astype(int)

    # Özel günler
    special_dates = ['2024-05-01', '2024-05-02', '2024-05-18', '2024-05-19', '2023-05-01', '2023-05-02',
                     '2023-05-18', '2023-05-19', '2022-05-01', '2022-05-02', '2022-05-18', '2022-05-19']
    df['special_day'] = df['date'].isin(pd.to_datetime(special_dates)).astype(int)

    # Lag ve Rolling
    df['orders_lag1'] = df['orders'].shift(1)
    df['orders_lag3'] = df['orders'].shift(3)
    df['orders_lag7'] = df['orders'].shift(7)
    df['orders_roll3'] = df['orders'].rolling(window=3).mean()

    df = pd.get_dummies(df, columns=['weekday'], drop_first=False)
    df.dropna(inplace=True)
    return df

# ---------------------------
# 2. Veriyi oku ve hazırla
# ---------------------------
df = pd.read_excel("EMT_new_data.xlsx")
df = feature_engineering(df)

# ---------------------------
# 3. Model girdi-çıktı ayır
# ---------------------------
X = df.drop(columns=['orders', 'date'])
y = df['orders']

# ---------------------------
# 4. Modeli eğit
# ---------------------------
model = GradientBoostingRegressor(random_state=42)
model.fit(X, y)

# ---------------------------
# 5. Modeli ve feature listesini kaydet
# ---------------------------
with open("order_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("order_model_features.pkl", "wb") as f:
    pickle.dump(list(X.columns), f)

print("✅ Model ve feature listesi başarıyla kaydedildi.")


✅ Model ve feature listesi başarıyla kaydedildi.
